In [308]:
from classes import *
def update(self):
    Grid=self.visible
    positions=np.where((Grid >= 1) & (Grid < 9))
    neighbors=self.neighbors(positions.T)

    
Grid.update=update

In [309]:
def arrange(nbomb,space):
    if(nbomb==1):
        return frozenset(frozenset({i}) for i in space)
    return frozenset(frozenset([i]) | t for i in space for t in arrange(nbomb-1,space - {i}))


In [34]:
class Barrangement:
    def __init__(self,arrangement,whole):
        self.arrangement=arrangement
        self.whole=whole
    def __add__(self,B1):
        intersect= B1.whole & self.whole
        return Barrangement({b1 | b2 if(b1 & b2 == b1 & intersect == b2 & intersect) else None
        for b1 in B1.arrangement
        for b2 in self.arrangement}-{None},B1.whole | self.whole)

In [ ]:
G=Grid(40,20)
G.generate(100)

GG=Game(G,Width=1200,Height=600,border=20,top_border=50)
GG.set_background_color((127,127,127))
GG.Grid.set_background_color((255,0,0))
GG.draw_board()
GG.play()

(array([14], dtype=int64), array([5], dtype=int64))
(array([14, 14], dtype=int64), array([4, 5], dtype=int64))
(array([14, 14, 15], dtype=int64), array([4, 5, 4], dtype=int64))
(array([12], dtype=int64), array([9], dtype=int64))
(array([11, 12], dtype=int64), array([9, 9], dtype=int64))
(array([14], dtype=int64), array([9], dtype=int64))
(array([14, 14], dtype=int64), array([ 9, 10], dtype=int64))
(array([ 0,  0,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,
        9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18,
       18, 19, 19, 19, 19, 19, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22

(array([ 0,  0,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,
        9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18,
       18, 19, 19, 19, 19, 19, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22,
       22, 22, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 25, 28, 29, 29, 29,
       29, 29, 29, 30, 30, 31, 31, 31, 32, 32, 33, 33, 33, 33, 33, 33, 33,
       34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 38, 38, 39, 39], dtype=int64), array([12, 14, 12, 14, 12, 14, 15, 16, 17,  7,  8,  9, 10, 11, 12, 17, 18,
       19,  7,  7,  8,  9, 11, 12, 13,  9, 11